Online BERT servislerinin denemesi

In [2]:
import numpy as np
from bert_serving.client import BertClient
from termcolor import colored

In [3]:
queries = ['makine öğrenmesi nedir', 'expert finding with deep learning', 'bert embedding word what accuracy']

In [ ]:
with BertClient() as bc:
    doc_vecs = bc.encode(queries)

In [ ]:
query_vec = bc.encode(['makine deep expert'])[0]
# compute normalized dot product as score
score = np.sum(query_vec * doc_vecs, axis=1) / np.linalg.norm(doc_vecs, axis=1)
topk_idx = np.argsort(score)[::-1][:topk]
print('top %d questions similar to "%s"' % (topk, colored(query, 'green')))
for idx in topk_idx:
    print('> %s\t%s' % (colored('%.1f' % score[idx], 'cyan'), colored(questions[idx], 'yellow')))

In [2]:
!pip install torch

In [ ]:
!pip install transformers
!pip install bert-serving-server
!pip install bert-serving-client
!pip install bert-extractive-summarizer
!pip install neuralcoref
python -m spacy download en_core_web_md

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BertTokenizer
torch.set_grad_enabled(False)

In [2]:
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

D:\Dev\Anaconda3\envs\bert\lib\site-packages\transformers\tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [3]:
# Tokens comes from a process that splits the input into sub-entities with interesting linguistic properties. 
tokens = tokenizer.tokenize("Bu örnek bir sorgu cümlesidir")
print("Tokens: {}".format(tokens))

# This is not sufficient for the model, as it requires integers as input, 
# not a problem, let's convert tokens to ids.
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Tokens id: {}".format(tokens_ids))

# Add the required special tokens
tokens_ids = tokenizer.build_inputs_with_special_tokens(tokens_ids)

# We need to convert to a Deep Learning framework specific format, let's use PyTorch for now.
tokens_pt = torch.tensor([tokens_ids])
print("Tokens PyTorch: {}".format(tokens_pt))

# Now we're ready to go through BERT with out input
outputs, pooled = model(tokens_pt)
print("Token wise output: {}, Pooled output: {}".format(outputs.shape, pooled.shape))

Tokens: ['Bu', 'örnek', 'bir', 'sorgu', 'cümlesi', '##dir']
Tokens id: [2123, 4073, 1996, 7643, 30773, 2067]
Tokens PyTorch: tensor([[    2,  2123,  4073,  1996,  7643, 30773,  2067,     3]])
Token wise output: torch.Size([1, 8, 768]), Pooled output: torch.Size([1, 768])


In [4]:
# tokens = tokenizer.tokenize("This is an input example")
# tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
# tokens_pt = torch.tensor([tokens_ids])

# This code can be factored into one-line as follow
tokens_pt2 = tokenizer.encode_plus("Bu örnek bir sorgu cümlesidir", return_tensors="pt")

for key, value in tokens_pt2.items():
    print("{}:\n\t{}".format(key, value))

outputs2, pooled2 = model(**tokens_pt2)
print("Difference with previous code: ({}, {})".format((outputs2 - outputs).sum(), (pooled2 - pooled).sum()))

input_ids:
	tensor([[    2,  2123,  4073,  1996,  7643, 30773,  2067,     3]])
token_type_ids:
	tensor([[0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
	tensor([[1, 1, 1, 1, 1, 1, 1, 1]])
Difference with previous code: (0.0, 0.0)


In [5]:
# Single segment input
single_seg_input = tokenizer.encode_plus("Bu örnek bir sorgu cümlesidir")

# Multiple segment input
multi_seg_input = tokenizer.encode_plus("This is segment A", "This is segment B")

print("Single segment token (str): {}".format(tokenizer.convert_ids_to_tokens(single_seg_input['input_ids'])))
print("Single segment token (int): {}".format(single_seg_input['input_ids']))
print("Single segment type       : {}".format(single_seg_input['token_type_ids']))

# Segments are concatened in the input to the model, with 
print()
print("Multi segment token (str): {}".format(tokenizer.convert_ids_to_tokens(multi_seg_input['input_ids'])))
print("Multi segment token (int): {}".format(multi_seg_input['input_ids']))
print("Multi segment type       : {}".format(multi_seg_input['token_type_ids']))

Single segment token (str): ['[CLS]', 'Bu', 'örnek', 'bir', 'sorgu', 'cümlesi', '##dir', '[SEP]']
Single segment token (int): [2, 2123, 4073, 1996, 7643, 30773, 2067, 3]
Single segment type       : [0, 0, 0, 0, 0, 0, 0, 0]

Multi segment token (str): ['[CLS]', 'Th', '##is', 'is', 'segment', 'A', '[SEP]', 'Th', '##is', 'is', 'segment', 'B', '[SEP]']
Multi segment token (int): [2, 9039, 2011, 2605, 22726, 37, 3, 9039, 2011, 2605, 22726, 38, 3]
Multi segment type       : [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]


In [6]:
# Padding highlight
tokens = tokenizer.batch_encode_plus(
    ["This is a sample", "This is another longer sample text"], 
    pad_to_max_length=True  # First sentence will have some PADDED tokens to match second sequence length
)

for i in range(2):
    print("Tokens (int)      : {}".format(tokens['input_ids'][i]))
    print("Tokens (str)      : {}".format([tokenizer.convert_ids_to_tokens(s) for s in tokens['input_ids'][i]]))
    print("Tokens (attn_mask): {}".format(tokens['attention_mask'][i]))
    print()

Tokens (int)      : [2, 9039, 2011, 2605, 69, 6844, 6882, 3, 0, 0, 0, 0, 0, 0]
Tokens (str)      : ['[CLS]', 'Th', '##is', 'is', 'a', 'sam', '##ple', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Tokens (attn_mask): [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]

Tokens (int)      : [2, 9039, 2011, 2605, 2085, 12837, 1972, 29178, 29561, 1972, 6844, 6882, 29238, 3]
Tokens (str)      : ['[CLS]', 'Th', '##is', 'is', 'an', '##oth', '##er', 'lo', '##ng', '##er', 'sam', '##ple', 'text', '[SEP]']
Tokens (attn_mask): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]



In [7]:
from transformers import TFBertModel, BertModel

# Let's load a BERT model for TensorFlow and PyTorch
model_tf = TFBertModel.from_pretrained('dbmdz/bert-base-turkish-cased')
model_pt = BertModel.from_pretrained('dbmdz/bert-base-turkish-cased')

ImportError: cannot import name 'TFBertModel'

In [35]:
from summarizer import Summarizer

body = 'Text body that you want to summarize with BERT Something else you want to summarize with BERT'
body2 = 'Something else you want to summarize with BERT'
model = Summarizer()
b1=model(body)
b2=model(body2)

In [ ]:
from summarizer import Summarizer
from summarizer.coreference_handler import CoreferenceHandler

#nlp = spacy.load(r'D:\Dev\Anaconda3\envs\bert\Lib\site-packages\en_core_web_md\en_core_web_md-2.3.0')

handler = CoreferenceHandler(greedyness=.4)
# How coreference works:
# >>>handler.process('''My sister has a dog. She loves him.''', min_length=2)
# ['My sister has a dog.', 'My sister loves a dog.']

body = 'Text body that you want to summarize with BERT'
body2 = 'Something else you want to summarize with BERT'
model = Summarizer(sentence_handler=handler)
model(body)
model(body2)

In [39]:
from transformers import AutoConfig
# Load model, model config and tokenizer via Transformers
model_name = "dbmdz/bert-base-turkish-cased"
custom_config = AutoConfig.from_pretrained(model_name)
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained(model_name)
custom_model = AutoModel.from_pretrained(model_name)

D:\Dev\Anaconda3\envs\bert\lib\site-packages\transformers\tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [27]:
body = 'Text body that you want to summarize with BERT'
body2 = 'Something else you want to summarize with BERT'
model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
model(body)
model(body2)

IndexError: index -2 is out of bounds for dimension 0 with size 1

In [38]:
from summarizer import Summarizer

body = '''
The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price.
The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal.
Mubadala, an Abu Dhabi investment fund, purchased 90% of the building for $800 million in 2008.
Real estate firm Tishman Speyer had owned the other 10%.
The buyer is RFR Holding, a New York real estate company.
Officials with Tishman and RFR did not immediately respond to a request for comments.
It's unclear when the deal will close.
The building sold fairly quickly after being publicly placed on the market only two months ago.
The sale was handled by CBRE Group.
The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.
The rent is rising from $7.75 million last year to $32.5 million this year to $41 million in 2028.
Meantime, rents in the building itself are not rising nearly that fast.
While the building is an iconic landmark in the New York skyline, it is competing against newer office towers with large floor-to-ceiling windows and all the modern amenities.
Still the building is among the best known in the city, even to people who have never been to New York.
It is famous for its triangle-shaped, vaulted windows worked into the stylized crown, along with its distinctive eagle gargoyles near the top.
It has been featured prominently in many films, including Men in Black 3, Spider-Man, Armageddon, Two Weeks Notice and Independence Day.
The previous sale took place just before the 2008 financial meltdown led to a plunge in real estate prices.
Still there have been a number of high profile skyscrapers purchased for top dollar in recent years, including the Waldorf Astoria hotel, which Chinese firm Anbang Insurance purchased in 2016 for nearly $2 billion, and the Willis Tower in Chicago, which was formerly known as Sears Tower, once the world's tallest.
Blackstone Group (BX) bought it for $1.3 billion 2015.
The Chrysler Building was the headquarters of the American automaker until 1953, but it was named for and owned by Chrysler chief Walter Chrysler, not the company itself.
Walter Chrysler had set out to build the tallest building in the world, a competition at that time with another Manhattan skyscraper under construction at 40 Wall Street at the south end of Manhattan. He kept secret the plans for the spire that would grace the top of the building, building it inside the structure and out of view of the public until 40 Wall Street was complete.
Once the competitor could rise no higher, the spire of the Chrysler building was raised into view, giving it the title.
'''

model = Summarizer()
result = model(body, min_length=60)
full = ''.join(result)
print(full)

The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price. The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal. It's unclear when the deal will close. The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building. Meantime, rents in the building itself are not rising nearly that fast.
